# 1. COnfig

Dựa trên code được cung cấp, có thể điều chỉnh số layer và số lượng nodes của một layer tại các vị trí sau:

1. Số layer được cấu hình trong `TreeBuilderConfig`:

```48:52:raptor/tree_builder.py
        if num_layers is None:
            num_layers = 5
        if not isinstance(num_layers, int) or num_layers < 1:
            raise ValueError("num_layers must be an integer and at least 1")
        self.num_layers = num_layers
```


Mặc định là 5 layers, bạn có thể thay đổi khi khởi tạo config:
```python
config = TreeBuilderConfig(num_layers=3)  # Thay đổi thành 3 layers
```

2. Số lượng nodes trong một layer được điều khiển bởi 2 tham số chính:

a. `top_k` trong `TreeBuilderConfig`:

```60:63:raptor/tree_builder.py
        if top_k is None:
            top_k = 5
        if not isinstance(top_k, int) or top_k < 1:
            raise ValueError("top_k must be an integer and at least 1")
```


b. `reduction_dimension` trong `ClusterTreeConfig`:

```20:22:raptor/cluster_tree_builder.py
        reduction_dimension=10,
        clustering_algorithm=RAPTOR_Clustering,  # Default to RAPTOR clustering
        clustering_params={},  # Pass additional params as a dict
```


Ví dụ cách cấu hình:
```python
config = ClusterTreeConfig(
    num_layers=3,           # Số layers
    top_k=3,               # Số nodes được chọn khi dùng mode "top_k"
    reduction_dimension=5   # Số nodes tối đa trong mỗi cluster
)
```

Lưu ý quan trọng về số nodes trong layer:
1. Khi sử dụng ClusterTreeBuilder, số nodes trong mỗi layer sẽ giảm dần khi đi lên trên (từ leaf nodes lên root)
2. Việc dừng tạo layer được kiểm soát bởi điều kiện:

```95:100:raptor/cluster_tree_builder.py
            if len(node_list_current_layer) <= self.reduction_dimension + 1:
                self.num_layers = layer
                logging.info(
                    f"Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: {layer}"
                )
                break
```


Nếu số nodes trong layer hiện tại nhỏ hơn hoặc bằng `reduction_dimension + 1`, quá trình tạo layer sẽ dừng lại.